In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./Scripts') 

# Recomendacion de productos, Content-Based
A continuación veremos paso a paso como se puede realizar un sistema de recomendacion basado en el contenido en python. 

http://www.p.valienteverde.com/sistemas-de-recomendacion-basados-en-el-contenido-content-based/

## Basado en el Contenido (ContendBased)

Por medio de la descripción del producto. Relaciones de tags

1. Vectorizacion del contenido, es decir, generar los tags
2. Ponderar los tags
3. Generar el motor de buscada de los productos similares

<img src='imagenes/contect_based.png'>

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import ContendBased as CB

In [4]:
datos = pd.read_csv("./BD/people_wiki.tar.gz", compression='gzip')
datos.head(3)

,people_wiki.csv,name,text
0,<http://dbpedia.org/resource/Gigi_Rigamonti>,Gigi Rigamonti,gigi rigamonti born 8 april 1949 is an italian...
1,<http://dbpedia.org/resource/Steve_Anderson_(m...,Steve Anderson (musician),steve anderson was briefly a member of the ban...
2,<http://dbpedia.org/resource/Tenzin_Delek_Rinp...,Tenzin Delek Rinpoche,lithang tulku tenzin delek rinpoche or tenzing...


In [5]:
entrada_elton_john=datos.query('name == "Elton John"')
entrada_elton_john

,people_wiki.csv,name,text
10,<http://dbpedia.org/resource/Elton_John>,Elton John,sir elton hercules john cbe born reginald kenn...


### Explorar los tags y su peso
1. Primero vamos a vertorizar (extración de los tags) la descripción del producto --> ```CountVectorizer()```
2. Calcular la importancia de cada tag en la descripcion del producto --> ```TfidfTransformer()```


##### Vectorizacion
Calculamos los tags

In [6]:
vectorizacion = CountVectorizer()
bag_of_words= vectorizacion.fit_transform(datos.text)

In [7]:
bag_words_elton_john = vectorizacion.transform(entrada_elton_john.text)
CB.mostrar_pesos_tags(bag_words_elton_john,vectorizacion)

,peso
tag_peso,
the,27
in,18
and,15
of,13
has,9
he,7
john,7
on,6


##### Normalizacion

In [8]:
tf_transformer = TfidfTransformer(use_idf=False).fit(bag_of_words)

In [9]:
matrix_elton_john_tf=tf_transformer.transform(bag_of_words[entrada_elton_john.index.values])
pesos_tf=CB.mostrar_pesos_tags(matrix_elton_john_tf,vectorizacion,descripcion='tf')
pesos_tf

,tf
tag_tf,
the,0.563848
in,0.375898
and,0.313249
of,0.271482
has,0.187949
he,0.146183
john,0.146183
on,0.125299


Fuck, los tags mas importante no describen los articulos
- **SOLUCION**: Normalizar y poderar teniendo en cuenta la frecuencia de cada tag en los demas documenos

In [10]:
tfidf_transformer = TfidfTransformer(use_idf=True).fit(bag_of_words)
matrix_elton_john_tf_idx=tfidf_transformer.transform(bag_of_words[entrada_elton_john.index.values])
pesos_tf_idx=CB.mostrar_pesos_tags(matrix_elton_john_tf_idx,vectorizacion,descripcion='tf-idx')
pesos_tf_idx

,tf-idx
tag_tf-idx,
the,0.245980
billboard,0.195711
john,0.189990
elton,0.186575
furnish,0.170638
in,0.164139
and,0.136804
songwriters,0.120128


Vamos por el buen camino, vamos a ayudar al algoritmo de dos formas:
1. Eliminando las palabras mas comunes --> ```stopwords```
2. Eliminado los numeros: Fuera de su contexto pierden todo su significado

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/pedro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
cachedStopWords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [13]:
vectorizacion_stop_words     = CountVectorizer(stop_words = cachedStopWords, token_pattern='(?u)\\b[a-zA-Z]\\w\\w+\\b')
bag_of_words_stop_words      = vectorizacion_stop_words.fit_transform(datos.text)
tfidf_transformer_stop_words = TfidfTransformer(use_idf=True).fit(bag_of_words_stop_words)

In [14]:
matrix_elton_john_tf_idx_stop_words=tfidf_transformer_stop_words.transform(bag_of_words_stop_words[entrada_elton_john.index.values])
pesos_tf_idx_filtrado=CB.mostrar_pesos_tags(matrix_elton_john_tf_idx_stop_words,vectorizacion_stop_words,descripcion='tf_idx_filtrado')
pesos_tf_idx_filtrado

,tf_idx_filtrado
tag_tf_idx_filtrado,
billboard,0.230409
john,0.223674
elton,0.219654
furnish,0.200891
songwriters,0.141425
award,0.141251
top,0.140106
aids,0.132142


##### Comparativa de las mejoras realizadas
- **tf**: Poderamos y normalizados los tags de cada articulo
- **tf-idx**: Poderamos y normalizados los tags de cada articulo teniendo en cuenta la frecuencia de cada tag en los demas articulos
- **tf_idx_filtrado**: Eliminamos las palabras mas comunes y los numeros a tf-idx

In [15]:
pd.concat([pesos_tf.reset_index(level=0),pesos_tf_idx.reset_index(level=0),pesos_tf_idx_filtrado.reset_index(level=0)],axis=1)

,tag_tf,tf,tag_tf-idx,tf-idx,tag_tf_idx_filtrado,tf_idx_filtrado
0,the,0.563848,the,0.245980,billboard,0.230409
1,in,0.375898,billboard,0.195711,john,0.223674
2,and,0.313249,john,0.189990,elton,0.219654
3,of,0.271482,elton,0.186575,furnish,0.200891
4,has,0.187949,furnish,0.170638,songwriters,0.141425
5,he,0.146183,in,0.164139,award,0.141251
6,john,0.146183,and,0.136804,top,0.140106
7,on,0.125299,songwriters,0.120128,aids,0.132142


##### Forma rapida
Todos los pasos anteriores se puede simplificar por medio de un metodo de sklearn que los agrutina

In [16]:
tfidf_vectorizer = TfidfVectorizer( stop_words=cachedStopWords, token_pattern='(?u)\\b[a-zA-Z]\\w\\w+\\b')
tfidf_vectorizer.fit(datos.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', '...aven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b[a-zA-Z]\\w\\w+\\b', tokenizer=None,
        use_idf=True, vocabulary=None)

In [17]:
CB.mostrar_pesos_tags(tfidf_vectorizer.transform(entrada_elton_john.text),tfidf_vectorizer)

,peso
tag_peso,
billboard,0.230409
john,0.223674
elton,0.219654
furnish,0.200891
songwriters,0.141425
award,0.141251
top,0.140106
aids,0.132142


### Prediccion
Una vez que ya hemos extraidos los tags de los productos, buscamos por similitud los mas parecidos

In [18]:
vecinos            = NearestNeighbors(n_neighbors=5,metric='cosine',algorithm='brute')
datos_por_tags     = tfidf_vectorizer.transform(datos.text)

In [19]:
vecinos.fit(datos_por_tags)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_neighbors=5, p=2, radius=1.0)

Con nuestro motor de recomendacion creado, podemos utilizarlo como un buscador de articulos. Como se verá, de los 5 actores propuestos, 4 de ellos al menos ha ganado un oscar !!!!

In [20]:
buscador  = tfidf_vectorizer.transform(['Award Actor Oscar'])
distancia,indices       = vecinos.kneighbors(buscador)
datos.iloc[indices[0],:]

,people_wiki.csv,name,text
185,<http://dbpedia.org/resource/Al_Pacino>,Al Pacino,alfredo james al pacino ptino born april 25 19...
10267,<http://dbpedia.org/resource/Ethan_Hawke>,Ethan Hawke,ethan green hawke born november 6 1970 is an a...
12795,<http://dbpedia.org/resource/Ralph_Fiennes>,Ralph Fiennes,ralph nathaniel twisletonwykehamfiennes ref fa...
3474,<http://dbpedia.org/resource/Daniel_Day-Lewis>,Daniel Day-Lewis,sir daniel michael blake daylewis kt born 29 a...
4677,<http://dbpedia.org/resource/Michael_Keaton>,Michael Keaton,michael john douglas born september 5 1951 bet...


Veamos que famosos nos relaciona con Al Pacino...

In [21]:
al_pacino_vectorizado  = tfidf_vectorizer.transform(datos.query('name == "Al Pacino"').text)
distancia,indices       = vecinos.kneighbors(al_pacino_vectorizado)
datos.iloc[indices[0],:]

,people_wiki.csv,name,text
185,<http://dbpedia.org/resource/Al_Pacino>,Al Pacino,alfredo james al pacino ptino born april 25 19...
12795,<http://dbpedia.org/resource/Ralph_Fiennes>,Ralph Fiennes,ralph nathaniel twisletonwykehamfiennes ref fa...
10267,<http://dbpedia.org/resource/Ethan_Hawke>,Ethan Hawke,ethan green hawke born november 6 1970 is an a...
3474,<http://dbpedia.org/resource/Daniel_Day-Lewis>,Daniel Day-Lewis,sir daniel michael blake daylewis kt born 29 a...
460,<http://dbpedia.org/resource/Jeff_Daniels>,Jeff Daniels,jeffrey warren jeff daniels born february 19 1...
